# Preliminaries

In [1]:
# install sent2vec
!pip install git+https://github.com/epfml/sent2vec

  Cloning https://github.com/epfml/sent2vec to /tmp/pip-req-build-bphcavvo
  Running command git clone -q https://github.com/epfml/sent2vec /tmp/pip-req-build-bphcavvo
  Created wheel for sent2vec: filename=sent2vec-0.0.0-cp36-cp36m-linux_x86_64.whl size=1139410 sha256=70900ee62cd62dee55729255e82a05bfd5c54e9950b5136a26cde40d3fa79756
  Stored in directory: /tmp/pip-ephem-wheel-cache-5eeecops/wheels/f5/1a/52/b5f36e8120688b3f026ac0cefe9c6544905753c51d8190ff17
Successfully built sent2vec


Write requirements to file, anytime you run it, in case you have to go back and recover dependencies.

Latest known such requirements are hosted for each notebook in the companion github repo, and can be pulled down and installed here if needed. Companion github repo is located at https://github.com/azunre/transfer-learning-for-nlp

In [2]:
!pip freeze > kaggle_image_requirements.txt

# Download IMDB Movie Review Dataset
Download IMDB dataset

In [3]:
import random
import pandas as pd

## Read-in the reviews and print some basic descriptions of them

!wget -q "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
!tar xzf aclImdb_v1.tar.gz

wget: /opt/conda/lib/libuuid.so.1: no version information available (required by wget)


# Define Tokenization, Stop-word and Punctuation Removal Functions
Before proceeding, we must decide how many samples to draw from each class. We must also decide the maximum number of tokens per email, and the maximum length of each token. This is done by setting the following overarching hyperparameters

**Preliminary overarching hyperparameters**


In [4]:
Nsamp = 1000 # number of samples to generate in each class - 'positive', 'negative'
maxtokens = 200 # the maximum number of tokens per document
maxtokenlen = 100 # the maximum length of each token

**Tokenization**

In [5]:
def tokenize(row):
    if row is None or row is '':
        tokens = ""
    else:
        tokens = str(row).split(" ")[:maxtokens]
    return tokens

**Use regular expressions to remove unnecessary characters**

Next, we define a function to remove punctuation marks and other nonword characters (using regular expressions) from the emails with the help of the ubiquitous python regex library. In the same step, we truncate all tokens to hyperparameter maxtokenlen defined above.

In [6]:
import re

def remove_reg_expressions(row):
    tokens = []
    try:
        for token in row:
            token = token.lower()
            token = re.sub(r'[\W\d]', " ", token)
            token = token[:maxtokenlen] # truncate token
            tokens.append(token)
    except:
        token = ""
        tokens.append(token)
    return tokens

**Stop-word removal**

Stop-words are also removed. Stop-words are words that are very common in text but offer no useful information that can be used to classify the text. Words such as is, and, the, are are examples of stop-words. The NLTK library contains a list of 127 English stop-words and can be used to filter our tokenized strings.

In [7]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')    

# these stopwords may indicate positivity/negativity of sentiment, so we remove them (keep them in corpus)
# stopwords.remove("no")
# stopwords.remove("nor")
# stopwords.remove("not")

# print(stopwords) # see default stopwords


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
def stop_word_removal(row):
    token = [token for token in row if token not in stopwords]
    token = filter(None, token)
    return token

# Assemble Embedding Vectors

The following functions are used to extract sent2vec embedding vectors for each review

In [9]:
import time
import sent2vec

s2v_model = sent2vec.Sent2vecModel()
start=time.time()
s2v_model.load_model('../input/sent2vec/wiki_unigrams.bin')
end = time.time()
print("Loading the sent2vec embedding took %d seconds"%(end-start))

Loading the sent2vec embedding took 42 seconds


In [10]:
def assemble_embedding_vectors(data):
    out = None
    for item in data:
        vec = s2v_model.embed_sentence(" ".join(str(i) for i in item if i))
        if vec is not None:
            if out is not None:
                out = np.concatenate((out,vec),axis=0)
            else:
                out = vec                                            
        else:
            pass
        
        
    return out

# Putting It All Together To Assemble Dataset

Now, putting all the preprocessing steps together we assemble our dataset...

In [11]:
import os
import numpy as np

# shuffle raw data first
def unison_shuffle_data(data, header):
    p = np.random.permutation(len(header))
    data = data[p]
    header = np.asarray(header)[p]
    return data, header

def load_data(path):
    data, sentiments = [], []
    for folder, sentiment in (('neg', 0), ('pos', 1)):
        folder = os.path.join(path, folder)
        for name in os.listdir(folder):
            with open(os.path.join(folder, name), 'r') as reader:
                  text = reader.read()
            text = tokenize(text)
            text = stop_word_removal(text)
            text = remove_reg_expressions(text)
            data.append(text)
            sentiments.append(sentiment)
    data_np = np.array(data)
    data, sentiments = unison_shuffle_data(data_np, sentiments)
    
    return data, sentiments

train_path = os.path.join('aclImdb', 'train')
test_path = os.path.join('aclImdb', 'test')
raw_data, raw_header = load_data(train_path)

print(raw_data.shape)
print(len(raw_header))

(25000,)
25000


In [12]:
# Subsample required number of samples
random_indices = np.random.choice(range(len(raw_header)),size=(Nsamp*2,),replace=False)
data_train = raw_data[random_indices]
header = raw_header[random_indices]

del raw_data, raw_header # huge and no longer needed, get rid of it

print("DEBUG::data_train::")
print(data_train)

DEBUG::data_train::
[list(['you', 'already', 'know', 'painful', 'watch', 'movie', 'is ', 'but', 'i', 'wonder', 'one', 'worst', 'movies', 'ever', 'include', 'one', 'beautiful', 'cars ', 'why', 'cars', 'victim', 'violation ', 'also', 'true', 'actors', 'performers', 'it ', 'so', 'earth', 'porsche ', 'lamborghini', 'whatever', 'could', 'allow', 'people', 'get', 'touch', 'cars', 'ruin', 'reputation', 'give', 'millions stop', 'getting', 'advantage', 'cars', 'earn', 'money', 'chests ', 'it', 'painful', 'love', 'cars ', 'it', 'painful', 'love', 'movies  br', '   br', '  i', 'want', 'money', 'back', '   '])
 list(['a', 'kid', 'ideals', 'tries', 'change', 'things', 'around', 'him ', 'a', 'boy', 'forced', 'become', 'man ', 'system ', 'a', 'system', 'hides', 'truth ', 'violating', 'rights', 'existence ', 'a', 'boy', 'who ', 'inspired', 'martin', 'luther', 'king ', 'stands', 'up ', 'tells', 'truth ', 'a', 'family', 'falling', 'apart ', 'fighting', 'it ', 'a', 'movie', 'can t', 'hide', 'from ', 'you

Display sentiments and their frequencies in the dataset, to ensure it is roughly balanced between classes

In [13]:
unique_elements, counts_elements = np.unique(header, return_counts=True)
print("Sentiments and their frequencies:")
print(unique_elements)
print(counts_elements)

Sentiments and their frequencies:
[0 1]
[ 979 1021]


**Featurize and Create Labels**

In [14]:
EmbeddingVectors = assemble_embedding_vectors(data_train)
print(EmbeddingVectors)

[[ 0.02330038 -0.14840275 -0.1021696  ... -0.255628   -0.09731507
   0.00285701]
 [ 0.02242229 -0.00493283  0.10543741 ...  0.19925834 -0.10933676
   0.12847123]
 [ 0.06502265 -0.01243187  0.02413149 ... -0.03431552  0.12365571
   0.1409206 ]
 ...
 [ 0.12317618  0.01931829 -0.10995793 ...  0.03704416 -0.06965005
   0.09366618]
 [ 0.09754995 -0.00553994  0.01447253 ... -0.10964575 -0.07332841
   0.12328903]
 [-0.18844478  0.07937658  0.10309957 ...  0.0677065  -0.07026131
   0.08437289]]


In [15]:
data = EmbeddingVectors

idx = int(0.7*data.shape[0])

# 70% of data for training
train_x = data[:idx,:]
train_y = np.array(header[:idx])
# # remaining 30% for testing
test_x = data[idx:,:]
test_y = np.array(header[idx:]) 

print("train_x/train_y list details, to make sure it is of the right form:")
print(len(train_x))
print(train_x)
print(train_y[:5])
print(len(train_y))

train_x/train_y list details, to make sure it is of the right form:
1400
[[ 0.02330038 -0.14840275 -0.1021696  ... -0.255628   -0.09731507
   0.00285701]
 [ 0.02242229 -0.00493283  0.10543741 ...  0.19925834 -0.10933676
   0.12847123]
 [ 0.06502265 -0.01243187  0.02413149 ... -0.03431552  0.12365571
   0.1409206 ]
 ...
 [ 0.202387   -0.11903944 -0.04019626 ...  0.12367007 -0.03494386
   0.08373439]
 [-0.07610612 -0.14455009 -0.02997797 ... -0.02688483  0.08298526
   0.18828106]
 [-0.02476747 -0.09346437 -0.00762459 ...  0.16376993 -0.1158855
   0.26556307]]
[0 1 0 1 1]
1400


# Train Shallow Model for IMDB Reviews

In [16]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout

input_shape = (len(train_x[0]),)
sent2vec_vectors = Input(shape=input_shape)
dense = Dense(512, activation='relu')(sent2vec_vectors)
dense = Dropout(0.1)(dense)
output = Dense(1, activation='sigmoid')(dense)
model = Model(inputs=sent2vec_vectors, outputs=output)

Using TensorFlow backend.


In [17]:
model.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
history = model.fit(train_x, train_y, validation_data=(test_x, test_y), batch_size=32,
                    nb_epoch=10, shuffle=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 1400 samples, validate on 600 samples
Epoch 1/10
1400/1400 [==============================] - 1s 410us/step - loss: 0.5963 - accuracy: 0.6857 - val_loss: 0.4629 - val_accuracy: 0.8067
Epoch 2/10
1400/1400 [==============================] - 0s 186us/step - loss: 0.4232 - accuracy: 0.8129 - val_loss: 0.4121 - val_accuracy: 0.8117
Epoch 3/10
1400/1400 [==============================] - 0s 187us/step - loss: 0.3708 - accuracy: 0.8329 - val_loss: 0.4072 - val_accuracy: 0.8000
Epoch 4/10
1400/1400 [==============================] - 0s 188us/step - loss: 0.3342 - accuracy: 0.8600 - val_loss: 0.4479 - val_accuracy: 0.7833
Epoch 5/10
1400/1400 [==============================] - 0s 192us/step - loss: 0.3065 - accuracy: 0.8671 - val_loss: 0.4331 - val_accuracy: 0.7967
Epoch 6/10
1400/1400 [==============================] - 0s 181us/step - loss: 0.2928 - accuracy: 0.8807 - val_loss: 0.4353 - val_accuracy: 0.7883
Epoch 7/10
1400/1400 [==============================] - 0s 175us/step - loss:

# Test Trained Model on Book Reviews from MDSD

In [18]:
!ls ../input/multi-domain-sentiment-dataset-books-and-dvds/

books.negative.review  dvd.negative.review
books.positive.review  dvd.positive.review


In [19]:
def parse_MDSD(data):
    out_lst = []
    for i in range(len(data)):
        txt = ""
        if(data[i]=="<review_text>\n"):
            j=i
            while(data[j]!="</review_text>\n"):
                txt = txt+data[j]
                j = j+1
            text = tokenize(txt)
            text = stop_word_removal(text)
            text = remove_reg_expressions(text)
            out_lst.append(text)
            
            #print(txt)
            #print(text)
            
    return out_lst

with open ("../input/multi-domain-sentiment-dataset-books-and-dvds/books.negative.review", "r", encoding="latin1") as myfile:
    data=myfile.readlines()
neg_books = parse_MDSD(data)
len(neg_books)

with open ("../input/multi-domain-sentiment-dataset-books-and-dvds/books.positive.review", "r", encoding="latin1") as myfile:
    data=myfile.readlines()
pos_books = parse_MDSD(data)
len(pos_books)

#print(neg_books)
#print(pos_books)

header = [0]*len(neg_books)
header.extend([1]*len(pos_books))
neg_books.extend(pos_books)
MDSD_data = np.array(neg_books)

data, sentiments = unison_shuffle_data(np.array(MDSD_data), header)

len(sentiments)

2000

**Try using the IMDB classifier directly on book review data...**

In [20]:
EmbeddingVectors = assemble_embedding_vectors(data)
print(EmbeddingVectors)
sentiments = np.asarray(sentiments)

[[-0.06236032 -0.12740827 -0.18496157 ...  0.06650508 -0.17147787
   0.12499873]
 [ 0.04192024 -0.03800137 -0.08864829 ...  0.03386298 -0.3149032
   0.0971027 ]
 [ 0.13418147 -0.08883765 -0.09897237 ...  0.06471071 -0.083598
   0.18522756]
 ...
 [-0.20760313 -0.11033539  0.20187528 ...  0.19972643 -0.2426406
  -0.01195074]
 [-0.04075969 -0.0933793  -0.0874037  ...  0.13744631 -0.08768537
   0.07796707]
 [-0.13110557 -0.18092139  0.06077183 ... -0.01214167 -0.06358923
  -0.08050677]]


In [21]:

print(model.evaluate(x=EmbeddingVectors,y=sentiments)) # evaluate IMDB classifier on books directly
print(model.metrics_names)

2000/2000 [==============================] - 0s 49us/step
[0.7492947010993958, 0.7214999794960022]
['loss', 'accuracy']


# Adaptation of Book Review Domain via Autoencoder

In [22]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

encoding_dim = 30 # chosen experimentally

input_shape = (len(train_x[0]),)
sent2vec_vectors = Input(shape=input_shape)
encoder = Dense(encoding_dim)(sent2vec_vectors)
dropout = Dropout(0.1)(encoder)
decoder = Dense(encoding_dim)(dropout)
dropout = Dropout(0.1)(decoder)
output = Dense(len(train_x[0]))(dropout)
autoencoder = Model(inputs=sent2vec_vectors, outputs=output)

In [23]:
autoencoder.compile(optimizer='adam',loss='mse',metrics=["mse","mae"])
autoencoder.fit(train_x,train_x,validation_data=(test_x, test_x), batch_size=32,
                    nb_epoch=50, shuffle=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 1400 samples, validate on 600 samples
Epoch 1/50
1400/1400 [==============================] - 0s 348us/step - loss: 0.0141 - mse: 0.0141 - mae: 0.0924 - val_loss: 0.0094 - val_mse: 0.0094 - val_mae: 0.0755
Epoch 2/50
1400/1400 [==============================] - 0s 90us/step - loss: 0.0091 - mse: 0.0091 - mae: 0.0751 - val_loss: 0.0084 - val_mse: 0.0084 - val_mae: 0.0717
Epoch 3/50
1400/1400 [==============================] - 0s 81us/step - loss: 0.0083 - mse: 0.0083 - mae: 0.0715 - val_loss: 0.0077 - val_mse: 0.0077 - val_mae: 0.0684
Epoch 4/50
1400/1400 [==============================] - 0s 95us/step - loss: 0.0078 - mse: 0.0078 - mae: 0.0692 - val_loss: 0.0073 - val_mse: 0.0073 - val_mae: 0.0668
Epoch 5/50
1400/1400 [==============================] - 0s 88us/step - loss: 0.0074 - mse: 0.0074 - mae: 0.0677 - val_loss: 0.0071 - val_mse: 0.0071 - val_mae: 0.0657
Epoch 6/50
1400/1400 [==============================] - 0s 92us/step - loss: 0.0072 - mse: 0.0072 - mae: 0.0666 - val

In [24]:
# transform EmbeddingVectors and sentiments with autoencoder.predict and then evaluate IDMB model again

EmbeddingVectorsProjected = autoencoder.predict(EmbeddingVectors)

print(EmbeddingVectorsProjected.shape)

print(model.evaluate(x=EmbeddingVectorsProjected,y=sentiments))
print(model.metrics_names)

(2000, 600)
2000/2000 [==============================] - 0s 53us/step
[0.6717412326335906, 0.7164999842643738]
['loss', 'accuracy']


In [25]:
from IPython.display import HTML
def create_download_link(title = "Download file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

#create_download_link(filename='GBMimportances.svg')

In [26]:
!rm -rf aclImdb
!rm aclImdb_v1.tar.gz